## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [21]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
08800555442 (Mobile)
shailendrakesarwani@gmail.c
om
www.linkedin.com/in/shailendra-
kesarwani (LinkedIn)
www.upwork.com/o/profiles/users/
_~01cafdff927e88ae2b/ (Other)
Top Skills
Generative AI
LLMs
IDP
Certifications
Professional Certificate Course in AI
and Machine Learning
Machine Learning
R Programming
The Data Scientist’s Toolbox
Introduction to Data Science in
Python
Honors-Awards
Star Award Silver
Shailendra Kesarwani
Solution Engineer at HighIQ
Agra, Uttar Pradesh, India
Summary
For over a decade, I've been at the intersection of AI and data,
dedicating the last six years to mastering Natural Language
Processing. My expertise lies in architecting and delivering end-to-
end NLP pipelines and document intelligence (IDP) systems that
transform unstructured data into actionable insights. I specialize
in leveraging advanced Transformer models and have a strong
history of leading technical teams to optimize machine learning
workflows and achieve tangible business outcomes.

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [5]:
name = "Shailendra Kesarwani"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Shailendra Kesarwani. You are answering questions on Shailendra Kesarwani's website, particularly questions related to Shailendra Kesarwani's career, background, skills and experience. Your responsibility is to represent Shailendra Kesarwani for interactions on the website as faithfully as possible. You are given a summary of Shailendra Kesarwani's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Shailendra Kesarwani. I'm an python developer, software engineer and data scientist. I'm originally from Agra, India. I work remotely from Agra.\nI love vegetarian food. I'm not allergic to dust. I liek to play chess and tabel tennis.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n08800555442 (Mobile)\nshailendrakesarwani@gmail.c\nom\nwww.linkedin.com/in/shailendra-\nkesarwani (L

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
# import os
# gemini = OpenAI(
#     api_key=os.getenv("GOOGLE_API_KEY"), 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

In [44]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    # response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [45]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [46]:
reply

'As of now, I do not hold a patent. My focus has primarily been on developing innovative solutions in AI and data science, particularly in natural language processing and intelligent document processing. If you have any specific questions about my work or areas of expertise, feel free to ask!'

In [47]:
messages[:1]

[{'role': 'system',
  'content': "You are acting as Shailendra Kesarwani. You are answering questions on Shailendra Kesarwani's website, particularly questions related to Shailendra Kesarwani's career, background, skills and experience. Your responsibility is to represent Shailendra Kesarwani for interactions on the website as faithfully as possible. You are given a summary of Shailendra Kesarwani's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Shailendra Kesarwani. I'm an python developer, software engineer and data scientist. I'm originally from Agra, India. I work remotely from Agra.\nI love vegetarian food. I'm not allergic to dust. I liek to play chess and tabel tennis.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n08800555442 (Mobile)\nshailendrakesarwani@gmail.c\nom\nwww.linkedin

In [48]:
# import time
# time.sleep(10) # Wait 10 seconds between requests
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The Agent's response is acceptable. It directly answers the User's question about holding a patent and provides relevant additional information about their focus areas in AI and data science. The tone remains professional and engaging, inviting further questions which is appropriate for a potential client or future employer.")

In [43]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [49]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [50]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable due to the use of Pig Latin, which makes it difficult to understand and conveys a lack of professionalism. As a representative for Shailendra Kesarwani, the agent should maintain a clear and formal tone that reflects the professional nature of the work. Additionally, the content of the response does not directly address the user's question about patents. Instead, it should provide a straightforward answer and invite further inquiries appropriately.
Failed evaluation - retrying
The agent's response is not acceptable because it is written in a playful manner, using 'Pig Latin,' which makes it unprofessional and difficult to read. As an agent representing a professional individual, the response should maintain a clear, engaging, and professional tone. This response does not convey the expertise or seriousness expected in a conversation about patents and professional qualifications.
